# Running DAX Queries Against a Power BI Semantic Model via REST API

This notebook demonstrates how to execute DAX queries against a Power BI dataset (semantic model) using the Power BI REST API. This requires a Premium or PPU workspace and an XMLA endpoint.

## Prerequisites
- Power BI workspace with Premium or PPU capacity
- Dataset published to the workspace
- Azure AD App Registration with Dataset.Read.All permission
- Client secret or certificate for authentication
- Python (requests, msal)

## 1. Get Access Token
Use MSAL to authenticate and obtain an OAuth2 token for Power BI REST API.

In [1]:
from msal import ConfidentialClientApplication
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('./../CODE/.env')

TENANT_ID = os.getenv('TENANT_ID')
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
AUTHORITY = f'https://login.microsoftonline.com/{TENANT_ID}/v2.0'
SCOPE = ['https://analysis.windows.net/powerbi/api/.default']

# print the environment variables
print("Environment Variables:")
print(f"TENANT_ID: {TENANT_ID}")
print(f"CLIENT_ID: {CLIENT_ID}")
print(f"CLIENT_SECRET: {CLIENT_SECRET}")

print("Acquiring token...")

app = ConfidentialClientApplication(
    CLIENT_ID, authority=AUTHORITY, client_credential=CLIENT_SECRET
)
token_response = app.acquire_token_for_client(scopes=SCOPE)
access_token = token_response['access_token']
print('Access token acquired.')

Environment Variables:
TENANT_ID: a172a259-b1c7-4944-b2e1-6d551f954711
CLIENT_ID: 20c5495d-b98c-410b-aa7b-9ea13dd70f61
CLIENT_SECRET: q-88Q~Zb_kYksiYkYI2VaCBM4JBF~7ozRfB5dcFU
Acquiring token...


ValueError: Unable to get authority configuration for https://login.microsoftonline.com/a172a259-b1c7-4944-b2e1-6d551f954711/v2.0. Authority would typically be in a format of https://login.microsoftonline.com/your_tenant or https://tenant_name.ciamlogin.com or https://tenant_name.b2clogin.com/tenant.onmicrosoft.com/policy.  Also please double check your tenant name or GUID is correct.

## 2. Set Workspace and Dataset IDs
Find these in the Power BI portal URL for your dataset.

In [ ]:
workspace_id = 'YOUR_WORKSPACE_ID'
dataset_id = 'YOUR_DATASET_ID'

## 3. Run a DAX Query Using the Execute Queries API
Replace the DAX query string with your own.

In [ ]:
import requests

dax_query = 'EVALUATE ROW("Test", 1+1)'  # Replace with your DAX
url = f'https://api.powerbi.com/v1.0/myorg/groups/{workspace_id}/datasets/{dataset_id}/executeQueries'
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}
body = {
    'queries': [{ 'query': dax_query }],
    'serializerSettings': { 'includeNulls': True }
}
response = requests.post(url, headers=headers, json=body)
print(response.json())

## 4. Parse and Display Results
The response will contain the results of your DAX query in JSON format.

In [ ]:
result = response.json()
# Example: print the first table
if 'results' in result and result['results'] and 'tables' in result['results'][0]:
    table = result['results'][0]['tables'][0]
    print('Columns:', [col['name'] for col in table['columns']])
    for row in table['rows']:
        print(row)
else:
    print('No results or error:', result)